In [1]:
import random
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium  #needed for interactive map
from folium.plugins import HeatMap
from shapely.geometry import Polygon, Point
%matplotlib inline
sns.set()


In [ ]:
bogota = gpd.read_file('/Users/SebastianGuzman/Desktop/poligonos-localidades/poligonos-localidades.shp')

dias = ["Lunes", "Martes", "Miercoles", "Jueves", "Viernes", "Sabado", "Domingo"]

horas = ["Mañana", "Tarde", "Noche"]

## Coordenadas ((latitud: Sur, Norte), (longitud: Oriente, Occidente))
square1 = ((4.6806, 4.7737), (-74.0210, -74.0768))
square2 = ((4.6806, 4.7497), (-74.0768, -74.1486))
square3 = ((4.6155, 4.6806), (-74.0278, -74.0978))
square4 = ((4.6155, 4.6806), (-74.0978, -74.1613))
square5 = ((4.5293, 4.6155), (-74.0614, -74.1116))
square6 = ((4.5293, 4.6155), (-74.1116, -74.1613))
square7 = ((4.5651, 4.6436), (-74.1613, -74.2046))
square8 = ((4.4896, 4.6021), (-74.0780, -74.1249))

city = (square1, square2, square3, square4, square5, square6, square7, square8)

In [ ]:
def get_coord():
    sq = random.choice(city)
    lat = round(random.choice(np.linspace(sq[0][0], sq[0][1],1000)), 6) 
    long = round(random.choice(np.linspace(sq[1][0], sq[1][1],1000)), 6)
    return lat, long

def selec_data(n):
    data = []
    i = 1
    while i < n:
        lat, long = get_coord()
        point = Point(long, lat)
        for polygon in bogota["geometry"]:
            if polygon.contains(point):
                data.append([lat, long, random.choice(dias), random.choice(horas), bogota.loc[bogota["geometry"]==polygon]["Nombre_de_l"].values[0]])
                i += 1
    return data

def generate_data():
    data = selec_data(1251)
    formatted_data = {
        "Latitud": [data[i][0] for i in range(len(data))],
        "Longitud": [data[i][1] for i in range(len(data))],
        "Hora": [data[i][3] for i in range(len(data))],
        "Dia": [data[i][2] for i in range(len(data))],
        "Localidad": [data[i][4] for i in range(len(data))]
    }
    final_data = pd.DataFrame(formatted_data, columns =["Latitud", "Longitud", "Hora", "Dia", "Localidad"])
    return final_data


In [ ]:
final_data = generate_data()
final_data.to_csv(path_or_buf="/Users/SebastianGuzman/Desktop/db_AppTaxis.csv")

In [2]:
bog_coords = [4.655085, -74.101300] # lat, long
folium_map = folium.Map(location=bog_coords, zoom_start=13, tiles="OpenStreetMap")

data = pd.read_csv("/Users/SebastianGuzman/Desktop/db_AppTaxis.csv", index_col=0)
my_zip = zip(data['Latitud'], data['Longitud'])
list_of_my_zip = list(my_zip)
for i in range(len(list_of_my_zip)):
    marker = folium.CircleMarker(location=[list_of_my_zip[i][0], list_of_my_zip[i][1]], radius = 2, color = "blue", fill = True)
    marker.add_to(folium_map)

folium_map